In [1]:
import math
from ipywidgets import interact
import ipywidgets as widgets

from ipyleaflet import Map, Marker, AwesomeIcon, ZoomControl, FullScreenControl, LayersControl, DrawControl, MeasureControl, ScaleControl, basemaps, TileLayer, Heatmap
from ipywidgets import Layout

import httpx
import json


In [2]:
def f(x=10):
    return math.factorial(x)


In [3]:
x_widget = widgets.IntSlider(min=0, max=100, step=1, value=10)
zoom_widget = widgets.IntSlider(min=0, max=15, step=1, value=10)
f_widget = widgets.Text(value='', disabled=True)

def update_f_widget_value(*args):
    f_widget.value = str(f(x_widget.value))

x_widget.observe(update_f_widget_value, 'value')


display(x_widget, f_widget)


IntSlider(value=10)

Text(value='', disabled=True)

In [4]:
# 37.871960, -122.259094

UCB_LOC = [37.871960, -122.259094]
MURLO_LOC = [43.1607, 11.3880]


def calculate_scale(m):
    # The Earth's circumference at the equator in meters
    earth_circumference = 40075000

    # Calculate the scale
    scale = earth_circumference / (256 * math.pow(2, m.zoom))

    return scale

def calculate_distance_per_cm(m):
    # Calculate the scale of the map in meters per pixel
    scale = calculate_scale(m)

    # Convert the scale to the distance represented by 1cm on the map
    distance_per_cm = scale * 37.8

    return distance_per_cm


def remove_heatmaps(m):
    for layer in m.layers:
        if isinstance(layer, Heatmap):
            m.remove(layer)

# https://central.isample.xyz/isamples_central/things_leaflet_heatmap?query=%2A%3A%2A&min_lat=-24.5&max_lat=49.4&min_lon=-124.73&max_lon=-66.95


def things_leaflet_heatmap(min_lat, max_lat, min_lon, max_lon, query='*:*'):
    base_url = "https://central.isample.xyz/isamples_central/things_leaflet_heatmap"
    r = httpx.get(base_url, params={'query': query, 'min_lat': min_lat, 'max_lat': max_lat, 'min_lon': min_lon, 'max_lon': max_lon})
    data = r.json()['data']
    return data

def heatmap_for_map(m, query='*:*', radius=2, blur=10):
    remove_heatmaps(m)
    data = things_leaflet_heatmap(m.bounds[0][0], m.bounds[1][0], m.bounds[0][1], m.bounds[1][1], query)
    heatmap = Heatmap(
        locations=data,  # Data points
        radius=radius,       # Radius of each “heat” point
        blur=blur,         # Amount of blur
    )
    m.add(heatmap)
    return heatmap

def make_map(center_x=37.871960, center_y=-122.259094, zoom=10, width='100%', height='500px',
             zoom_control=False, scroll_wheel_zoom=True,
             controls=(ZoomControl(position='topleft'), FullScreenControl(), 
                LayersControl(position='topright'), DrawControl(), MeasureControl(), ScaleControl(position='bottomleft'))
             ):
         
    m = Map(center=[center_x, center_y], zoom=zoom, layout=Layout(width=width, height=height),
            zoom_control=zoom_control, scroll_wheel_zoom=scroll_wheel_zoom, controls=controls)

    google_map = TileLayer(url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', name='Google Maps', attribution='Google')
    google_satellite = TileLayer(
        url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        attribution="Google",
        name="Google Satellite"
    )

    # m.add(google_map)
    # m.add(google_satellite)

    # TO DO: turn on only the basemap
    
    return m


m = make_map(MURLO_LOC[0], MURLO_LOC[1], 6)
display(m)


def handle_bounds_change(*args):
    radius = 2*m.zoom
    blur = 5*m.zoom

    distance_per_cm = calculate_distance_per_cm(m)
    # print(m.zoom, 1000000/distance_per_cm)

    heatmap_for_map(m, query='*:*', radius=radius, blur=blur)
    
m.observe(handle_bounds_change, 'bounds')



Map(center=[43.1607, 11.388], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [ ]:
m.bounds

In [ ]:

zoom_widget = widgets.IntSlider(min=0, max=18, step=1, value=10)

center = (37.8715,-122.2730)

m = make_map(center[0], center[1], zoom=10)

icon = AwesomeIcon(
    name='SFO',
    marker_color='green',
    icon_color='darkgreen',
    spin=False
)


marker = Marker(location=center, draggable=False, icon=icon, title='SFO Airport')
m.add_layer(marker);

def update_zoom_value_value(*args):
    m.zoom = str((zoom_widget.value))

zoom_widget.observe(update_zoom_value_value, 'value')

marker.location = (37.6213, -122.3790)
display(zoom_widget, m)

# Now that the marker is on the Map, you can drag it with your mouse,
# it will automatically update the `marker.location` attribute in Python

# You can also update the marker location from Python, that will update the
# marker location on the Map:


In [ ]:
m.zoom, m.center, m.bounds

In [ ]:
# how to draw a marker on the map
from ipyleaflet import Map, Marker

center = (52.204793, 360.121558)


In [ ]:
%%bash

curl -X 'GET' \
  'https://central.isample.xyz/isamples_central/things_leaflet_heatmap?query=%2A%3A%2A&min_lat=-24.5&max_lat=49.4&min_lon=-124.73&max_lon=-66.95' \
  -H 'accept: application/json'